# --- Day 24: Blizzard Basin ---

[Puzzle Description](https://adventofcode.com/2022/day/24)

In [1]:
import numpy as np
from functools import cache

Important assumptions/information about the input:
* There are no vertical blizzards in the columns containing the start and the finish positions.
* The start position is at [0, 1] and the end position is at [-1, -2].

In [2]:
blizzards = list()
with open("day_24_input.txt") as file:
    while line := file.readline().rstrip():
        blizzards.append(
            [
                [1 if c == ">" else 0 for c in line],
                [1 if c == "v" else 0 for c in line],
                [1 if c == "<" else 0 for c in line],
                [1 if c == "^" else 0 for c in line],
            ]
        )
# reorganise the blizzard information
blizzards = np.array(blizzards)
blizzards = np.moveaxis(blizzards, 1, -1)
blizzards = blizzards[1:-1, 1:-1, :]
blizzards

array([[[0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        ...,
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 1, 0]],

       [[1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0]],

       [[1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        ...,
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]],

       ...,

       [[0, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 1, 0]],

       [[0, 0, 1, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [1, 0, 0, 0]],

       [[0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        ...,
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 0]]])

In [3]:
@cache
def get_blizzards(i, j, time):
    result = sum(
        [
            blizzards[i, (j - time) % blizzards.shape[1], 0],
            blizzards[(i - time) % blizzards.shape[0], j, 1],
            blizzards[i, (j + time) % blizzards.shape[1], 2],
            blizzards[(i + time) % blizzards.shape[0], j, 3],
        ]
    )
    return result

## Part One

In [4]:
# Storing the information for the BFS
cycle_length = blizzards.shape[0] * blizzards.shape[1]
seen = np.zeros((blizzards.shape[0], blizzards.shape[1], cycle_length))
seen_start = np.zeros(cycle_length)

In [5]:
queue = [(-1, 0, 0)]
seen_start[0] = 1
while queue:
    # Get the next configuration in queue:
    config = queue.pop(0)
    cur_y, cur_x, t = config[0], config[1], config[2]

    # Special info for the start
    if cur_y < 0:
        # Consider not moving
        if not seen_start[(t + 1) % cycle_length]:
            queue.append((cur_y, cur_x, t + 1))
            seen_start[(t + 1) % cycle_length] = 1
        # Consider moving out
        if (
            not seen[cur_y + 1, cur_x, (t + 1) % cycle_length]
            and get_blizzards(cur_y + 1, cur_x, (t + 1) % cycle_length) == 0
        ):
            queue.append((cur_y + 1, cur_x, t + 1))
            seen[cur_y + 1, cur_x, (t + 1) % cycle_length] = 1
        continue

    # If the finish is within reach, stop exploring!
    if cur_y == blizzards.shape[0] - 1 and cur_x == blizzards.shape[1] - 1:
        print(f"Part One: {t + 1}")
        break

    # Stay put if possible:
    if (
        seen[cur_y, cur_x, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y, cur_x, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y, cur_x, t + 1))
        seen[cur_y, cur_x, (t + 1) % cycle_length] = 1

    # Explore right if possible:
    if (
        cur_x < blizzards.shape[1] - 1
        and seen[cur_y, cur_x + 1, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y, cur_x + 1, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y, cur_x + 1, t + 1))
        seen[cur_y, cur_x + 1, (t + 1) % cycle_length] = 1

    # Explore down if possible:
    if (
        cur_y < blizzards.shape[0] - 1
        and seen[cur_y + 1, cur_x, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y + 1, cur_x, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y + 1, cur_x, t + 1))
        seen[cur_y + 1, cur_x, (t + 1) % cycle_length] = 1

    # Explore left if possible:
    if (
        cur_x > 0
        and seen[cur_y, cur_x - 1, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y, cur_x - 1, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y, cur_x - 1, t + 1))
        seen[cur_y, cur_x - 1, (t + 1) % cycle_length] = 1

    # Explore up if possible
    if (
        cur_y > 0
        and seen[cur_y - 1, cur_x, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y - 1, cur_x, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y - 1, cur_x, t + 1))
        seen[cur_y - 1, cur_x, (t + 1) % cycle_length] = 1

Part One: 242


## Part Two

 Lazy on the Christmas Eve, the solution is to repeat *Part One* twice more!

*Moving back to the start:*

In [6]:
# Resetting the information for the BFS
seen = np.zeros((blizzards.shape[0], blizzards.shape[1], cycle_length))
seen_finish = np.zeros(cycle_length)

# Starting at the end at result time t + 1 from Task 1:
queue = [(blizzards.shape[0], blizzards.shape[1] - 1, t + 1)]
seen_finish[t + 1] = 1
while queue:
    # Get the next configuration in queue:
    config = queue.pop(0)
    cur_y, cur_x, t = config[0], config[1], config[2]

    # Special info for the start
    if cur_y > blizzards.shape[0] - 1:
        # Consider not moving
        if not seen_finish[(t + 1) % cycle_length]:
            queue.append((cur_y, cur_x, t + 1))
            seen_finish[(t + 1) % cycle_length] = 1
        # Consider moving out
        if (
            not seen[cur_y - 1, cur_x, (t + 1) % cycle_length]
            and get_blizzards(cur_y - 1, cur_x, (t + 1) % cycle_length) == 0
        ):
            queue.append((cur_y - 1, cur_x, t + 1))
            seen[cur_y - 1, cur_x, (t + 1) % cycle_length] = 1
        continue

    # If the start is within reach, stop exploring!
    if cur_y == 0 and cur_x == 0:
        print(f"Time back at the start: {t + 1}")
        break

    # Stay put if possible:
    if (
        seen[cur_y, cur_x, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y, cur_x, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y, cur_x, t + 1))
        seen[cur_y, cur_x, (t + 1) % cycle_length] = 1

    # Explore right if possible:
    if (
        cur_x < blizzards.shape[1] - 1
        and seen[cur_y, cur_x + 1, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y, cur_x + 1, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y, cur_x + 1, t + 1))
        seen[cur_y, cur_x + 1, (t + 1) % cycle_length] = 1

    # Explore down if possible:
    if (
        cur_y < blizzards.shape[0] - 1
        and seen[cur_y + 1, cur_x, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y + 1, cur_x, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y + 1, cur_x, t + 1))
        seen[cur_y + 1, cur_x, (t + 1) % cycle_length] = 1

    # Explore left if possible:
    if (
        cur_x > 0
        and seen[cur_y, cur_x - 1, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y, cur_x - 1, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y, cur_x - 1, t + 1))
        seen[cur_y, cur_x - 1, (t + 1) % cycle_length] = 1

    # Explore up if possible
    if (
        cur_y > 0
        and seen[cur_y - 1, cur_x, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y - 1, cur_x, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y - 1, cur_x, t + 1))
        seen[cur_y - 1, cur_x, (t + 1) % cycle_length] = 1

Time back at the start: 478


*Moving to the finish point again:*

In [7]:
# Resetting the information for the BFS
seen = np.zeros((blizzards.shape[0], blizzards.shape[1], cycle_length))
seen_start = np.zeros(cycle_length)

# Starting at the end at result time t + 1 from Task 1:
queue = [(-1, 0, t + 1)]
seen_start[t + 1] = 1
while queue:
    # Get the next configuration in queue:
    config = queue.pop(0)
    cur_y, cur_x, t = config[0], config[1], config[2]

    # Special info for the start
    if cur_y < 0:
        # Consider not moving
        if not seen_start[(t + 1) % cycle_length]:
            queue.append((cur_y, cur_x, t + 1))
            seen_start[(t + 1) % cycle_length] = 1
        # Consider moving out
        if (
            not seen[cur_y + 1, cur_x, (t + 1) % cycle_length]
            and get_blizzards(cur_y + 1, cur_x, (t + 1) % cycle_length) == 0
        ):
            queue.append((cur_y + 1, cur_x, t + 1))
            seen[cur_y + 1, cur_x, (t + 1) % cycle_length] = 1
        continue

    # If the finish is within reach, stop exploring!
    if cur_y == blizzards.shape[0] - 1 and cur_x == blizzards.shape[1] - 1:
        print(f"Part Two: {t + 1}")
        break

    # Stay put if possible:
    if (
        seen[cur_y, cur_x, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y, cur_x, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y, cur_x, t + 1))
        seen[cur_y, cur_x, (t + 1) % cycle_length] = 1

    # Explore right if possible:
    if (
        cur_x < blizzards.shape[1] - 1
        and seen[cur_y, cur_x + 1, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y, cur_x + 1, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y, cur_x + 1, t + 1))
        seen[cur_y, cur_x + 1, (t + 1) % cycle_length] = 1

    # Explore down if possible:
    if (
        cur_y < blizzards.shape[0] - 1
        and seen[cur_y + 1, cur_x, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y + 1, cur_x, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y + 1, cur_x, t + 1))
        seen[cur_y + 1, cur_x, (t + 1) % cycle_length] = 1

    # Explore left if possible:
    if (
        cur_x > 0
        and seen[cur_y, cur_x - 1, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y, cur_x - 1, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y, cur_x - 1, t + 1))
        seen[cur_y, cur_x - 1, (t + 1) % cycle_length] = 1

    # Explore up if possible
    if (
        cur_y > 0
        and seen[cur_y - 1, cur_x, (t + 1) % cycle_length] == 0
        and get_blizzards(cur_y - 1, cur_x, (t + 1) % cycle_length) == 0
    ):
        queue.append((cur_y - 1, cur_x, t + 1))
        seen[cur_y - 1, cur_x, (t + 1) % cycle_length] = 1

Part Two: 720
